In [1]:
# Decision Trees
# Treat All Participants as One
# Try with 10% ... 90% for Training (several iterations for each)

In [2]:
use_key_features = False
shuffle_times = 100
iterations = 10
output_file = 'decision_tree_results_all_af.csv'

In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn import metrics
from itertools import combinations
import itertools
import json
pd.options.mode.chained_assignment = None

# 1 -- Load Data

In [4]:
# Windowed Data: 1 Second Overlapping Windows, Feature Median + Variance in Window 
p5_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p5_master_window.csv'
p7_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p7_master_window.csv'
p9_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p9_master_window.csv'
p11_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p11_master_window.csv'
p12_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p12_master_window.csv'
p17_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p17_master_window.csv'
p18_file = '../../../../../Google Drive File Stream/My Drive/USC Expeditions Year 5/Analysis/Engagement/Data/Master Window/p18_master_window.csv'

data5 = pd.read_csv(p5_file)
data7 = pd.read_csv(p7_file)
data9 = pd.read_csv(p9_file)
data11 = pd.read_csv(p11_file)
data12 = pd.read_csv(p12_file)
data17 = pd.read_csv(p17_file)
data18 = pd.read_csv(p18_file)

In [5]:
data = [data5, data7, data9, data11, data12, data17, data18]
all_data = pd.concat(data, ignore_index=True, sort=True)

In [6]:
remove = [
'engagement_change',
'ros_GAME_STATE',
'ros_PARTICIPANT_STATE',
'ros_ROBOT_STATE',
'ros_activity',
'ros_diff_1_change',
'ros_diff_2_change',
'ros_diff_3_change',
'ros_diff_4_change',
'ros_diff_5_change',
'ros_difficulty',
'ros_skill_EM_change',
'ros_skill_NC_change',
'ros_skill_OS_change',
'ros_ts_attempt_var',
'ros_games_session_change',
'ros_in_game_change',
'ros_mistakes_session_change',
'ros_mistakes_game_change',
'ros_skill',
'ros_ts_game_start_var',
'ros_ts_robot_talked_var',
'ros_game_correct',
'ros_game_incorrect',
'ros_game_start',
'ros_mistake_made',
]

all_data = all_data.drop(columns=remove)

In [7]:
# Remove rows where engagment NaN
all_data = all_data[np.isfinite(all_data['engagement'])]

# Remove rows where engagment is -1
all_data = all_data[all_data['engagement']>=0]

# 2 -- Choose Feature Set

Feature Dictionary: https://docs.google.com/spreadsheets/d/1ewoVPHwW68Ins0AOVZf-0lsl_wW0_ZzuByuDiNJETBY/edit?usp=sharing

### Data Overview

In [8]:
# Main Columns
basic_cols = []
for i in all_data.columns:
    if 'of_' not in i and 'op_' not in i and 'ros_' not in i and 'a_' not in i:
        basic_cols.append(i)
        
basic_cols = sorted(basic_cols)
for i in basic_cols:
    print(i)

engagement
participant
session_num
timestamp


In [9]:
# Open Face Columns

of_cols = []
for i in all_data.columns:
    if ('of_' in i or 'op_' in i) and '_change' not in i and '_var' not in i:
        of_cols.append(i)
        
of_cols = sorted(of_cols)
for i in of_cols:
    print(i)

of_AU01_c
of_AU02_c
of_AU04_c
of_AU05_c
of_AU06_c
of_AU07_c
of_AU09_c
of_AU10_c
of_AU12_c
of_AU14_c
of_AU15_c
of_AU17_c
of_AU20_c
of_AU23_c
of_AU25_c
of_AU26_c
of_AU28_c
of_AU45_c
of_confidence
of_gaze_0_x
of_gaze_0_y
of_gaze_0_z
of_gaze_1_x
of_gaze_1_y
of_gaze_1_z
of_gaze_angle_x
of_gaze_angle_y
of_gaze_distance
of_gaze_distance_x
of_gaze_distance_y
of_pose_Rx
of_pose_Ry
of_pose_Rz
of_pose_Tx
of_pose_Ty
of_pose_Tz
of_pose_distance
of_success
of_ts_success
op_num_people


In [10]:
# Audio Columns

a_cols = []
for i in all_data.columns:
    if 'a_' in i and '_change' not in i and '_var' not in i:
        a_cols.append(i)
        
a_cols = sorted(a_cols)
for i in a_cols:
    print(i)

a_harmonicity
a_intensity
a_mfcc_0
a_mfcc_1
a_pitch_frequency
a_pitch_strength


In [11]:
# ROS Columns

ros_cols = []
for i in all_data.columns:
    if 'ros_' in i and '_change' not in i and '_var' not in i:
        ros_cols.append(i)
        
ros_cols = sorted(ros_cols)
for i in ros_cols:
    print(i)

ros_aptitude
ros_diff_1
ros_diff_2
ros_diff_3
ros_diff_4
ros_diff_5
ros_games_session
ros_in_game
ros_mistakes_game
ros_mistakes_session
ros_skill_EM
ros_skill_NC
ros_skill_OS
ros_ts_attempt
ros_ts_game_start
ros_ts_robot_talked


In [12]:
# For Window Only:
non_window_features = []
window_features = []
for i in all_data.columns:
    if i not in basic_cols:
        if 'change' in i or 'var' in i:
            window_features.append(i)
        else:
            non_window_features.append(i)
        
window_features = sorted(window_features)
for i in window_features:
    print(i)

a_harmonicity_var
a_intensity_var
a_mfcc_0_var
a_mfcc_1_var
a_pitch_frequency_var
a_pitch_strength_var
of_AU01_c_change
of_AU02_c_change
of_AU04_c_change
of_AU05_c_change
of_AU06_c_change
of_AU07_c_change
of_AU09_c_change
of_AU10_c_change
of_AU12_c_change
of_AU14_c_change
of_AU15_c_change
of_AU17_c_change
of_AU20_c_change
of_AU23_c_change
of_AU25_c_change
of_AU26_c_change
of_AU28_c_change
of_AU45_c_change
of_confidence_var
of_gaze_0_x_var
of_gaze_0_y_var
of_gaze_0_z_var
of_gaze_1_x_var
of_gaze_1_y_var
of_gaze_1_z_var
of_gaze_angle_x_var
of_gaze_angle_y_var
of_gaze_distance_var
of_gaze_distance_x_var
of_gaze_distance_y_var
of_pose_Rx_var
of_pose_Ry_var
of_pose_Rz_var
of_pose_Tx_var
of_pose_Ty_var
of_pose_Tz_var
of_pose_distance_var
of_success_change
of_ts_success_var
op_num_people_change
ros_aptitude_var


In [13]:
# Columns where NaNs filled with max value
nan_max_cols = ['of_gaze_0_x',
'of_gaze_0_y',
'of_gaze_0_z',
'of_gaze_1_x',
'of_gaze_1_y',
'of_gaze_1_z',
'of_gaze_angle_x',
'of_gaze_angle_y',
'of_gaze_distance',
'of_gaze_distance_x',
'of_gaze_distance_y',
'of_pose_Rxv',
'of_pose_Ry',
'of_pose_Rz',
'of_pose_Tx',
'of_pose_Ty',
'of_pose_Tz',
'of_pose_distance']

In [14]:
# Key Features (from feature analysis) 
# Note: Timestamp automatically included

key_features = [
'op_num_people',
'of_pose_distance',
'of_gaze_distance',
'of_confidence',
'ros_mistakes_session',
'ros_ts_robot_talked']

### Filter Feature Set (Optional)

In [15]:
# always include basic_cols, add desired group of features 
# features_to_keep = all_data.columns

if use_key_features:
    features_to_keep = basic_cols + key_features
else:
    features_to_keep = basic_cols + of_cols + ros_cols + a_cols + window_features

all_data = all_data[features_to_keep]

In [16]:
# All as One Participant: Shuffle Data! 

for i in range(shuffle_times):
    all_data.reindex(np.random.permutation(all_data.index))
all_data = all_data.reset_index(drop=True)

# 3 -- Scenario Based Modeling

- Open Face Success / Failure
- Robot Talking / Not Talking
- First 10 Minutes / After 10 Minutes

In [17]:
# Create Separate Models for Different Scenarios

# for i,d in enumerate(data):
#     # Open Face Success/Failure
#     data[i] = d.loc[d['of_success']==1]
#     data[i] = d.loc[d['of_success']==0]
    
#     # Robot Talking/Not Talking
#     data[i] = d.loc[d['ros_ts_robot_talked']==0]
#     data[i] = d.loc[d['ros_ts_robot_talked']>0]
    
#     # First 10 Min/After 10 Min
#     data[i] = d.loc[d['timestamp']<=(10*60)]
#     data[i] = d.loc[d['timestamp']>(10*60)]

# 4 -- Modeling

In [18]:
# Function: Formulate Train-Test Split 
# Includes Preprocessing 

# split_size: how much data for training
def split(split_size):
    y_data = all_data['engagement'].copy()
    X_data = all_data.drop(columns=['engagement', 'session_num', 'participant'], axis=1).copy()
        
    X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=1-split_size, shuffle=True)

    # Preprocessing: Standardization
    # x' = ( x - mean(x) ) / ( stdev(x) )
    # => x' is z-score, NaN's filled with min_val
    for c in X_train.columns:
        mean = np.nanmean(X_train[c])
        std = np.nanstd(X_train[c])

        if std == 0:
            X_train[c] = (X_train[c]-mean)
            X_test[c] = (X_test[c]-mean)
        else:
            X_train[c] = (X_train[c]-mean)/(std)
            X_test[c] = (X_test[c]-mean)/(std)

        if c not in nan_max_cols:
            min_val = np.nanmin(X_train[c])

            X_train[c] = X_train[c].fillna(min_val)
            X_test[c] = X_test[c].fillna(min_val)
        else:
            max_val = np.nanmax(X_train[c])

            X_train[c] = X_train[c].fillna(max_val)
            X_test[c] = X_test[c].fillna(max_val) 

    return X_train, y_train, X_test, y_test

In [19]:
# MODEL HERE

# Inputs: X_train, y_train, X_test, y_test, hp, isVerbose
# hp = Hyperparameter Dictionary, isVerbose = whether to return all accuracy metrics

# Output: Accuracy Metrics as Dictionary 

# IMPORTS HERE
from sklearn.tree import DecisionTreeClassifier

# Hyperparameter Combinations to Try:  
hp_names = ['max_depth', 'min_samples_split', 'min_samples_leaf', 'max_features']
hp_list = [[25, 50, 100, 250, 500, 1000, None], [0.1, 0.2, 0.5, 2],  [0.1, 0.2, 0.5, 1], [None, 'sqrt']]

def model(X_train, y_train, X_test, y_test, hp, isVerbose):
    # Model Here: must create pred & scores arrays
    clf = DecisionTreeClassifier(max_depth=hp['max_depth'], min_samples_split=hp['min_samples_split'], min_samples_leaf=hp['min_samples_leaf'], max_features=hp['max_features'])
    clf.fit(X_train.values, y_train.values)
    
    pred = clf.predict(X_test.values)
    scores = clf.predict_proba(X_test.values)[:,1]    
    
    # Evaluation
    results = {}
    accuracy = accuracy_score(y_test, pred)
    results['accuracy'] = accuracy
    
    if isVerbose:
        try:
            auc = roc_auc_score(y_test, scores)
        except:
            auc = np.nan
        results['auc'] = auc
    
        precision = metrics.precision_score(y_test, pred, average=None)
        recall = metrics.recall_score(y_test, pred, average=None)
        f1 = metrics.f1_score(y_test, pred, average=None)

        results['precision_0'], results['precision_1'] = precision[0], precision[1]
        results['recall_0'], results['recall_1'] = recall[0], recall[1]
        results['f1_0'], results['f1_1'] = f1[0], f1[1]
    
    return results

In [ ]:
all_results = []
for perc in tqdm([0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]):
    print('Training Percentage:', perc)
    for n in range(iterations):
        print()
        new_results = {}
        
        # Shuffle Data For This Iteration
        for i in range(shuffle_times):
            all_data.reindex(np.random.permutation(all_data.index))        
        all_data = all_data.reset_index(drop=True)

        # Train - Test Split (and preprocessing)
        X_train, y_train, X_test, y_test = split(perc)
        X_train = X_train.reset_index(drop=True)
        y_train = y_train.reset_index(drop=True)

        # Set Up Cross Validation Groups
        div = int(len(X_train)/3)
        X_cv1 = X_train.loc[:div]
        X_cv2 = X_train.loc[div+1:div*2]
        X_cv3 = X_train.loc[(div*2)+1:]
        
        y_cv1 = y_train.loc[:div]
        y_cv2 = y_train.loc[div+1:div*2]
        y_cv3 = y_train.loc[(div*2)+1:]
        
        cv1_X_train = X_cv1.append(X_cv2)
        cv1_X_test = X_cv3
        cv1_y_train = y_cv1.append(y_cv2)
        cv1_y_test = y_cv3
        
        cv2_X_train = X_cv1.append(X_cv3)
        cv2_X_test = X_cv2
        cv2_y_train = y_cv1.append(y_cv3)
        cv2_y_test = y_cv2

        cv3_X_train = X_cv3.append(X_cv2)
        cv3_X_test = X_cv1
        cv3_y_train = y_cv3.append(y_cv2)
        cv3_y_test = y_cv1

        # Hyperparameter Combinations        
        hp_combos = list(itertools.product(*hp_list))
        best, best_acc = None, 0
        for h in hp_combos:
            hp = {}
            for i,v in enumerate(h):
                hp[hp_names[i]] = v
            
            # Cross-Validation
            res1 = model(cv1_X_train, cv1_y_train, cv1_X_test, cv1_y_test, hp, False)
            res2 = model(cv2_X_train, cv2_y_train, cv2_X_test, cv2_y_test, hp, False)
            res3 = model(cv3_X_train, cv3_y_train, cv3_X_test, cv3_y_test, hp, False)
            
            acc = np.mean([res1['accuracy'], res2['accuracy'], res3['accuracy']])
            print(hp, acc)
            if acc > best_acc:
                best = hp
                best_acc = acc
        
        # Evaluate on Test Using Best Hyperaparemeters  
        print('Best HP', best)
        results = model(X_train, y_train, X_test, y_test, best, True)
        print(results)
        results['train_percentage'] = perc
        results['iteration'] = n
        results['hyperparameters'] = json.dumps(best)
        all_results.append(results)

  0%|          | 0/11 [00:00<?, ?it/s]

Training Percentage: 0.01

{'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 25} 0.7931214635271914
{'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'max_features': 'sqrt', 'max_depth': 25} 0.7485556209899885
{'min_samples_leaf': 0.2, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 25} 0.7859463604407706
{'min_samples_leaf': 0.2, 'min_samples_split': 0.1, 'max_features': 'sqrt', 'max_depth': 25} 0.7398520438653587
{'min_samples_leaf': 0.5, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 25} 0.6833547399996339
{'min_samples_leaf': 0.5, 'min_samples_split': 0.1, 'max_features': 'sqrt', 'max_depth': 25} 0.6610794814475257
{'min_samples_leaf': 1, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 25} 0.770870792733624
{'min_samples_leaf': 1, 'min_samples_split': 0.1, 'max_features': 'sqrt', 'max_depth': 25} 0.7526187911378215
{'min_samples_leaf': 0.1, 'min_samples_split': 0.2, 'max_features': None, 'max_depth': 25}

{'min_samples_leaf': 1, 'min_samples_split': 0.2, 'max_features': None, 'max_depth': 100} 0.7668702475071022
{'min_samples_leaf': 1, 'min_samples_split': 0.2, 'max_features': 'sqrt', 'max_depth': 100} 0.7732404073755662
{'min_samples_leaf': 0.1, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': 100} 0.8074489126555449
{'min_samples_leaf': 0.1, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': 100} 0.7462050475867835
{'min_samples_leaf': 0.2, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': 100} 0.7740055733461085
{'min_samples_leaf': 0.2, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': 100} 0.7263277525556133
{'min_samples_leaf': 0.5, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': 100} 0.6833547399996339
{'min_samples_leaf': 0.5, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': 100} 0.684945829896213
{'min_samples_leaf': 1, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': 100} 0.8074489126555449


{'min_samples_leaf': 1, 'min_samples_split': 2, 'max_features': None, 'max_depth': 500} 0.7557439740959399
{'min_samples_leaf': 1, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 500} 0.7374596721789285
{'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 1000} 0.7931214635271914
{'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'max_features': 'sqrt', 'max_depth': 1000} 0.7509441046747917
{'min_samples_leaf': 0.2, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 1000} 0.7859463604407706
{'min_samples_leaf': 0.2, 'min_samples_split': 0.1, 'max_features': 'sqrt', 'max_depth': 1000} 0.7382362755808303
{'min_samples_leaf': 0.5, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 1000} 0.6833547399996339
{'min_samples_leaf': 0.5, 'min_samples_split': 0.1, 'max_features': 'sqrt', 'max_depth': 1000} 0.6571017567060778
{'min_samples_leaf': 1, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 1000} 0.7700733445677

{'min_samples_leaf': 0.5, 'min_samples_split': 0.2, 'max_features': None, 'max_depth': 25} 0.6611040329543031
{'min_samples_leaf': 0.5, 'min_samples_split': 0.2, 'max_features': 'sqrt', 'max_depth': 25} 0.6356465946090365
{'min_samples_leaf': 1, 'min_samples_split': 0.2, 'max_features': None, 'max_depth': 25} 0.7613356998484494
{'min_samples_leaf': 1, 'min_samples_split': 0.2, 'max_features': 'sqrt', 'max_depth': 25} 0.7271706694957544
{'min_samples_leaf': 0.1, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': 25} 0.7589071760909469
{'min_samples_leaf': 0.1, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': 25} 0.722357522866253
{'min_samples_leaf': 0.2, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': 25} 0.7462087633925343
{'min_samples_leaf': 0.2, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': 25} 0.7374445008159366
{'min_samples_leaf': 0.5, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': 25} 0.6611040329543031
{'min_s

...

{'min_samples_leaf': 0.5, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': None} 0.6813267569913943
{'min_samples_leaf': 0.5, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': None} 0.674525253501149
{'min_samples_leaf': 1, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': None} 0.7943797056780694
{'min_samples_leaf': 1, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': None} 0.7542062758592043
{'min_samples_leaf': 0.1, 'min_samples_split': 2, 'max_features': None, 'max_depth': None} 0.8030806258277977
{'min_samples_leaf': 0.1, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': None} 0.7513623907448199
{'min_samples_leaf': 0.2, 'min_samples_split': 2, 'max_features': None, 'max_depth': None} 0.766218550398913
{'min_samples_leaf': 0.2, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': None} 0.7477625293919558
{'min_samples_leaf': 0.5, 'min_samples_split': 2, 'max_features': None, 'max_depth': None} 0.6813267569913943


{'min_samples_leaf': 0.5, 'min_samples_split': 2, 'max_features': None, 'max_depth': 50} 0.6832956064421326
{'min_samples_leaf': 0.5, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 50} 0.6594704567891684
{'min_samples_leaf': 1, 'min_samples_split': 2, 'max_features': None, 'max_depth': 50} 0.861491165338156
{'min_samples_leaf': 1, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 50} 0.8443180409478693
{'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 100} 0.8039457379445102
{'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'max_features': 'sqrt', 'max_depth': 100} 0.7627879849999778
{'min_samples_leaf': 0.2, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 100} 0.7658903605206713
{'min_samples_leaf': 0.2, 'min_samples_split': 0.1, 'max_features': 'sqrt', 'max_depth': 100} 0.7465394976927234
{'min_samples_leaf': 0.5, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 100} 0.6832956064421326
{'min_samp

{'min_samples_leaf': 1, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 500} 0.8035380362879523
{'min_samples_leaf': 1, 'min_samples_split': 0.1, 'max_features': 'sqrt', 'max_depth': 500} 0.7696092759591554
{'min_samples_leaf': 0.1, 'min_samples_split': 0.2, 'max_features': None, 'max_depth': 500} 0.8039457379445102
{'min_samples_leaf': 0.1, 'min_samples_split': 0.2, 'max_features': 'sqrt', 'max_depth': 500} 0.7695000958813444
{'min_samples_leaf': 0.2, 'min_samples_split': 0.2, 'max_features': None, 'max_depth': 500} 0.7658903605206713
{'min_samples_leaf': 0.2, 'min_samples_split': 0.2, 'max_features': 'sqrt', 'max_depth': 500} 0.7476133546286071
{'min_samples_leaf': 0.5, 'min_samples_split': 0.2, 'max_features': None, 'max_depth': 500} 0.6832956064421326
{'min_samples_leaf': 0.5, 'min_samples_split': 0.2, 'max_features': 'sqrt', 'max_depth': 500} 0.6534545070646094
{'min_samples_leaf': 1, 'min_samples_split': 0.2, 'max_features': None, 'max_depth': 500} 0.8038562450708163

{'min_samples_leaf': 0.1, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': None} 0.7950359789402667
{'min_samples_leaf': 0.1, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': None} 0.7465495763831432
{'min_samples_leaf': 0.2, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': None} 0.7535400410799662
{'min_samples_leaf': 0.2, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': None} 0.7524461718209198
{'min_samples_leaf': 0.5, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': None} 0.6832956064421326
{'min_samples_leaf': 0.5, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': None} 0.6534545070646094
{'min_samples_leaf': 1, 'min_samples_split': 0.5, 'max_features': None, 'max_depth': None} 0.7948172171321009
{'min_samples_leaf': 1, 'min_samples_split': 0.5, 'max_features': 'sqrt', 'max_depth': None} 0.7516209401827808
{'min_samples_leaf': 0.1, 'min_samples_split': 2, 'max_features': None, 'max_depth': None} 0.8039457

{'min_samples_leaf': 0.1, 'min_samples_split': 2, 'max_features': None, 'max_depth': 50} 0.8035777749719055
{'min_samples_leaf': 0.1, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 50} 0.7770474221660005
{'min_samples_leaf': 0.2, 'min_samples_split': 2, 'max_features': None, 'max_depth': 50} 0.7666162846056007
{'min_samples_leaf': 0.2, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 50} 0.738325788604738
{'min_samples_leaf': 0.5, 'min_samples_split': 2, 'max_features': None, 'max_depth': 50} 0.6834546851740778
{'min_samples_leaf': 0.5, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 50} 0.6524998893186198
{'min_samples_leaf': 1, 'min_samples_split': 2, 'max_features': None, 'max_depth': 50} 0.8572550590648963
{'min_samples_leaf': 1, 'min_samples_split': 2, 'max_features': 'sqrt', 'max_depth': 50} 0.831390952903888
{'min_samples_leaf': 0.1, 'min_samples_split': 0.1, 'max_features': None, 'max_depth': 100} 0.8035777749719055
{'min_samples_leaf': 0.

In [ ]:
results = pd.DataFrame(columns=['train_percentage', 'iteration', 'accuracy', 'auc', 'precision_0', 'precision_1', 'recall_0', 'recall_1', 'f1_0', 'f1_1', 'hyperparameters'])
results = results.append(all_results, ignore_index=True, sort=True)

In [ ]:
results.head()

In [ ]:
results.to_csv(output_file, index=False)